In [ ]:
#Question 1a
import pandas as pd
import numpy as np


df1 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT')
df2 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT')
df3 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT')
df4 = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT')

df1['source'] = '17-18'
df2['source'] = '15-16'
df3['source'] = '13-14'
df4['source'] = '11-12'

DEMO_df = df1.append(df2).append(df3).append(df4)

DEMO_df = DEMO_df.loc[:, ['SEQN','RIAGENDR','RIDAGEYR',
                          'RIDRETH3','DMDEDUC2',
                          'DMDMARTL', 'RIDSTATR', 
                          'SDMVPSU', 'SDMVSTRA', 
                          'WTMEC2YR', 'WTINT2YR',
                          'source']]
DEMO_df = DEMO_df.rename(columns = {'SEQN':'id',
                                    'RIAGENDR': 'gender',
                                    'RIDAGEYR': 'age',
                                    'RIDRETH3': 'race and ethnicity',
                                    'DMDEDUC2': 'education', 
                                    'DMDMARTL': 'marital status'})

#DEMO_df = DEMO_df.replace(to_replace = np.nan, value = -1)
DEMO_df['gender'].replace(to_replace = [1, 2, np.nan], 
                          value = ['male', 'female',
                                  'missing'], inplace = True)

DEMO_df['race and ethnicity'].replace(to_replace = [1, 2, 3, 4, 6, 7, np.nan], 
                                      value = ['mexican american','other hispanic',
                                               'non-hispanic white', 'non-hispanic black',
                                               'non-hispanic asian', 'other race', 'missing'], inplace = True)
DEMO_df['education'].replace(to_replace = [1, 2, 3, 4, 5, 7, 9, np.nan], 
                             value = ['less than 9th grade', '9-11th grade','high school graduate',
                                        'some college or AA degree','clooege graduate or above', 
                                      'refused','dont know', 'missing'], inplace = True)
DEMO_df['marital status'].replace(to_replace = [1, 2, 3, 4, 5, 6, 77, 99, np.nan], 
                                 value = ['married', 'widowed', 'divorced', 'separated', 'never married', 
                                          'living with partner', 'refused', 'dont know', 'missing'], inplace = True)
DEMO_df.to_pickle('PS4_DEMO_df.pickle')

In [ ]:
#Question 1a
demo = pd.read_pickle('PS4_DEMO_df.pickle')

In [ ]:
#Question 1b
den = pd.read_pickle('DEN_df.pickle')
den = den.loc[:,['SEQN', 'OHDDESTS']]

den = den.rename(columns = {'SEQN':'id'})

# den['OHDDESTS'].replace(to_replace = [1, 2, 3, np.nan], 
#                             value = ['complete', 'partial',
#                                     'not done', 'mission'],
#                            inplace = True)

demo_ohdd = pd.merge(demo, den, on = 'id', how = 'left')

demo_ohdd['under_20'] = 'no'
demo_ohdd.loc[demo_ohdd.age < 20, 'under_20'] = 'yes'

demo_ohdd = demo_ohdd.loc[:, ['id', 'gender', 'age', 'under_20',
                             'education', 'RIDSTATR', 'OHDDESTS']]
demo_ohdd.rename(columns = {'RIDSTATR': 'exam_status',
                           'OHDDESTS':'ohx_status'}, inplace = True)

demo_ohdd['college'] = np.nan

demo_ohdd.loc[demo_ohdd.education == 'missing', 'college'] = 'No college/ <20'
demo_ohdd.loc[demo_ohdd.education == 'refused', 'college'] = 'No college/ <20'
demo_ohdd.loc[demo_ohdd.education == 'dont know', 'college'] = 'No college/ <20'
demo_ohdd.loc[demo_ohdd.education == 'less than 9th grade', 'college'] = 'No college/ <20'
demo_ohdd.loc[demo_ohdd.education == '9-11th grade', 'college'] = 'No college/ <20'
demo_ohdd.loc[demo_ohdd.education == 'high school graduate', 'college'] = 'No college/ <20'

demo_ohdd.loc[demo_ohdd.education == 'some college or AA degree', 'college'] = 'some college/college graduate'
demo_ohdd.loc[demo_ohdd.education == 'clooege graduate or above', 'college'] = 'some college/college graduate'

demo_ohdd['ohx'] = np.where(demo_ohdd['ohx_status'] == 2, 'complete', 'missing')
demo_ohdd.loc[demo_ohdd.exam_status == 2, 'ohx'] = 'complete'
demo_ohdd['ohx'].astype('category')
demo_ohdd

In [ ]:
#Question 1d
dropped_demo_ohdd = demo_ohdd.drop(['id', 'education',	'exam_status', 'ohx_status'], 1)

age_mean = dropped_demo_ohdd.groupby('ohx').mean()
age_sd = dropped_demo_ohdd.groupby('ohx').std(ddof=1)

age_com = '{} ({})'.format(round(age_mean.iloc[0,0]), round(age_sd.iloc[0,0]))
age_mis = '{} ({})'.format(round(age_mean.iloc[1,0]), round(age_sd.iloc[1,0]))